In [2]:
%tensorflow_version 2.x

In [3]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Loading the dataset using Keras

train_path = tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

test_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
csv_cols = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']

species = ['Setosa','Versicolor','Virginica']

In [6]:
train = pd.read_csv(train_path,names=csv_cols,header=0)

test = pd.read_csv(test_path,names=csv_cols,header=0)

In [7]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [8]:
test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [9]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [10]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [11]:
train.shape

(120, 4)

In [12]:

train_y.shape

(120,)

In [13]:
def input_fn(features,labels,training = True,batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [14]:
feature_cols = []
for key in train.keys():
  feature_cols.append(tf.feature_column.numeric_column(key=key))
print(feature_cols)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [15]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_cols,
    hidden_units = [30,10],
    n_classes = 3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpkvk5ptsv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
classifier.train(
    input_fn=lambda: input_fn(train,train_y,training=True),steps = 50000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpkvk5ptsv/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving checkpoints for 10000 into /tmp/tmpkvk5ptsv/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...
INFO:tensorflow:loss = 0.30858213, step = 10000
INFO:tensorflow:global_step/sec: 454.267
INFO:tensorflow:loss = 0.31120986, step = 10100 (0.225 sec)
INFO:tensorflow:global_step/sec: 577.347
INFO:tensorflow:loss = 0.29680005, step = 10200 (0.173 sec)
INFO:tensorflow:global_step/sec: 549.415
INFO:tensorflow:loss = 0.30233893, step = 10300 (0.179 sec)
INFO:tensorflow:global_step/sec: 550.437
INFO:tensorflow:loss = 0.29368976, step = 10400 (0.182 sec)

In [21]:
eval_result = classifier.evaluate(
    input_fn = lambda: input_fn(test,test_y,training=False)
)
print('\n Test Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-04T16:18:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpkvk5ptsv/model.ckpt-60000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18309s
INFO:tensorflow:Finished evaluation at 2021-10-04-16:18:17
INFO:tensorflow:Saving dict for global step 60000: accuracy = 0.93333334, average_loss = 0.087712415, global_step = 60000, loss = 0.087712415
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60000: /tmp/tmpkvk5ptsv/model.ckpt-60000

 Test Accuracy: 0.933

